In [1]:
import numpy as np
import scipy.sparse as sp
import opt_einsum as oe
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic as tpreiodic_old
from quant_rotor.core.dense.t_amplitudes_periodic_Test import t_periodic as tperiodic_Test
from quant_rotor.core.dense.t_amplitudes_periodic_new import t_periodic as tpreiodic_dense_new

# Testing the unique element step.

In [2]:
site = 5
state = 11
g = 0.1

In [3]:
one_max_s, two_max_s, energy_s, t_a_i_tensor_s, t_ab_ij_tensor_s = tpreiodic_dense_new(site, state, g, fast=True)

Done.


In [4]:
one_max_d, two_max_d, energy_d, t_a_i_tensor_d, t_ab_ij_tensor_d = tpreiodic_old(site, state, g)

In [36]:
t_a_i_tensor_s

array([ 0.            ,  0.            , -0.004009756177, -0.004009756177,  0.            ,  0.            ,  0.000000471072,  0.000000471072,  0.            ,  0.            ])

In [35]:
t_a_i_tensor_d[0]

array([[ 0.            +0.j],
       [ 0.            +0.j],
       [-0.004010522975+0.j],
       [-0.004010522975+0.j],
       [ 0.            +0.j],
       [ 0.            +0.j],
       [ 0.000000470726+0.j],
       [ 0.000000470726+0.j],
       [ 0.            +0.j],
       [ 0.            +0.j]])

In [5]:
if np.array_equal(one_max_d, one_max_s):
    print(f"One Max: {np.array_equal(one_max_d, one_max_s)}")
else:
    print(f"Delta One Max: {np.abs(one_max_d - one_max_s)}")

if np.array_equal(two_max_d, two_max_s):
    print(f"Two Max: {np.array_equal(two_max_d, two_max_s)}")
else:
    print(f"Delta Two Max: {np.abs(two_max_d - two_max_s)}")

if np.array_equal(energy_d, energy_s):
    print(f"Energy: {np.array_equal(energy_d, energy_s)}")
else:
    print(f"DeltaEnergy: {np.abs(energy_d - energy_s)}")

print(f"t_1: {np.allclose(t_a_i_tensor_d[0].reshape(state-1), t_a_i_tensor_s, 1e-10)}") # 
diff = np.abs(t_a_i_tensor_s - t_a_i_tensor_d[0].reshape(state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

print(f"t_2: {np.allclose(t_ab_ij_tensor_d[0].reshape(site, state-1, state-1), t_ab_ij_tensor_s, 1e-10)}") #
diff = np.abs(t_ab_ij_tensor_s - t_ab_ij_tensor_d[0].reshape(site, state-1, state-1))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff)

Delta One Max: 3.1333428975060016e-10
Delta Two Max: 1.9056113963133895e-08
DeltaEnergy: 0.13066900501794015
t_1: False
Max absolute difference: 7.667985042012632e-07
t_2: False
Max absolute difference: 0.0002193857790231806


# Check einsums.

In [6]:
p = state
a = state - 1
i = 1

A_old = np.random.rand(a, p).astype(np.float64)
B_old = np.random.rand(p, i).astype(np.float64)

B_new = B_old.reshape(p).astype(np.float64)
BB = (oe.contract("q,s->qs", B_new, B_new).reshape(p**2)).astype(np.float64)
AA =(oe.contract("ap,bq->abpq", A_old, A_old).reshape(a**2, p**2)).astype(np.float64)

In [7]:
A_old_sp = sp.csr_matrix(A_old)

B_new_sp = sp.csr_matrix(B_new.astype(np.float64))
BB_sp = sp.csr_matrix(BB)
AA_sp = sp.csr_matrix(AA)

## Residual single.

Term 1.

In [8]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

H_pq_old = H_pq_new = np.random.rand(p, p).astype(np.longdouble)
H_pq_old_sp = sp.csr_matrix(H_pq_old)

In [9]:
R_new += oe.contract("ap,pq,q->a", A_old, H_pq_new, B_new)
R_old += oe.contract("ap,pq,qi->ai", A_old, H_pq_old, B_old)
R_new_dot += A_old @ (H_pq_old @ B_new)
R_new_dot_sp += (A_old_sp @ (H_pq_old_sp @ B_new_sp.T)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 2.

In [10]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

T_cd_old = np.random.rand(a, a, i, i).astype(np.longdouble)
T_cd_new = T_cd_old.reshape(a**2)
T_cd_new_sp = sp.csr_matrix(T_cd_new)

V_cd_old = np.random.rand(p, i, a, a).astype(np.longdouble)
V_cd_new = V_cd_old.reshape(p, a**2)
V_cd_new_sp = sp.csr_matrix(V_cd_new)

In [11]:
R_old += oe.contract("ap,plcd,cdil->ai", A_old, V_cd_old, T_cd_old)
R_new += oe.contract("ap,pC,C->a", A_old, V_cd_new, T_cd_new)
R_new_dot += A_old @ (V_cd_new @ T_cd_new)
R_new_dot_sp += (A_old_sp @ (V_cd_new_sp @ T_cd_new_sp.T)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 3.

In [12]:
R_new = np.zeros((a), dtype=np.float64)
R_old = np.zeros((a, i), dtype=np.float64)
R_new_dot = np.zeros((a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, i), dtype=np.float64))

V_qq_old = np.random.rand(p, i, p, p)
V_qq_new = V_qq_old.reshape(p, p**2)
V_qq_new_sp = sp.csr_matrix(V_qq_new)

In [13]:
R_old += oe.contract("ap,plqs,qi,sl->ai", A_old, V_qq_old, B_old, B_old)
R_new += oe.contract("ap,pQ,Q->a", A_old, V_qq_new, BB)
R_new_dot += A_old @ (V_qq_new @ BB)
R_new_dot_sp += (A_old_sp @ (V_qq_new_sp @ BB_sp.T)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a)))

diff = np.abs(R_new - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a)))

diff = np.abs(R_new_dot - R_old.reshape(a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old))

diff1 = np.abs(R_new_dot_sp - R_old)
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 1.4210854715202004e-14 

Dot:
Equal: True
Max absolute difference: 1.4210854715202004e-14 

Dot Sparse:
Equal: True
Max absolute difference: 4.263256414560601e-14


## Residual doulbe sym.

Term 1.

In [14]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_pqrs_old = np.random.rand(p, p, p, p).astype(np.float64)
V_pqrs_new = V_pqrs_old.reshape(p**2, p**2)
V_pqrs_new_sp = sp.csr_matrix(V_pqrs_new)

In [15]:
R_old += oe.contract("ap,bq,pqrs,ri,sj->abij", A_old, A_old, V_pqrs_old, B_old, B_old)
R_new += oe.contract("BQ,QR,R->B", AA, V_pqrs_new, BB).reshape(a, a)
R_new_dot += (AA @ (V_pqrs_new @ BB)).reshape(a, a)
R_new_dot_sp += ((AA_sp @ (V_pqrs_new_sp @ BB_sp.T)).reshape(a, a)).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 1.7053025658242404e-13 

Dot:
Equal: True
Max absolute difference: 1.7053025658242404e-13 

Dot Sparse:
Equal: True
Max absolute difference: 3.410605131648481e-13


Term 2.

In [16]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_0y_old = np.random.rand(a, a, i, i).astype(np.float64)
T_0y_new = T_0y_old.reshape(a**2)
T_0y_new_sp = sp.csr_matrix(T_0y_new)

V_pqcd_old = np.random.rand(p, p, a, a).astype(np.float64)
V_pqcd_new = V_pqcd_old.reshape(p**2, a**2)
V_pqcd_new_sp = sp.csr_matrix(V_pqcd_new)

In [17]:
R_old += oe.contract("ap,bq,pqcd,cdij->abij", A_old, A_old, V_pqcd_old, T_0y_old)
R_new += oe.contract("BQ,QC,C->B", AA, V_pqcd_new, T_0y_new).reshape(a, a)
R_new_dot += (AA @ (V_pqcd_new @ T_0y_new)).reshape(a, a)
R_new_dot_sp += (AA_sp @ (V_pqcd_new_sp @ T_0y_new_sp.T)).reshape(a, a).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 3.410605131648481e-13 

Dot:
Equal: True
Max absolute difference: 3.410605131648481e-13 

Dot Sparse:
Equal: True
Max absolute difference: 1.0231815394945443e-12


Term 3.

In [18]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_klpq_old = np.random.rand(i, i, p, p)
V_klpq_new = V_klpq_old.reshape(p, p)
V_klpq_new_sp = sp.csr_matrix(V_klpq_new)

T_0y_new_3 = T_0y_old.reshape(a, a)
T_0y_new_3_sp = sp.csr_matrix(T_0y_new_3)

In [19]:
R_old -= oe.contract("abkl,klpq,pi,qj->abij", T_0y_old, V_klpq_old, B_old, B_old)
R_new -= oe.contract("ab,pq,p,q->ab", T_0y_new_3, V_klpq_new, B_new, B_new)
R_new_dot -= T_0y_new_3 * ((V_klpq_new @ B_new) @ B_new)
R_new_dot_sp -= (T_0y_new_3_sp * ((V_klpq_new_sp @ B_new_sp.T).T @ B_new_sp.T)[0 ,0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 1.7763568394002505e-15


Term 4.

In [20]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_0y_new_3 = T_0y_old.reshape(a, a)
T_0y_new_3_sp = sp.csr_matrix(T_0y_new_3)

V_klcd_old = np.random.rand(i, i, a, a)
V_klcd_new = V_klcd_old.reshape(a, a)
V_klcd_new_sp = sp.csr_matrix(V_klcd_new)

In [21]:
R_old -= oe.contract("abkl,klcd,cdij->abij", T_0y_old, V_klcd_old, T_0y_old)
R_new -= oe.contract("ab,cd,cd->ab", T_0y_new_3, V_klcd_new, T_0y_new_3)
R_new_dot -= T_0y_new_3 * np.sum(V_klcd_new * T_0y_new_3)
R_new_dot_sp -= (T_0y_new_3_sp * np.sum(V_klcd_new_sp.multiply(T_0y_new_3_sp))).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 5.

In [22]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

V_klcd_old = np.random.rand(i, i, a, a)
V_klcd_new = V_klcd_old.reshape(a, a)
V_klcd_new_sp = sp.csr_matrix(V_klcd_new)

T_0z_old = np.random.rand(a, a, i, i)
T_0z_new = T_0z_old.reshape(a, a)
T_0z_new_sp = sp.csr_matrix(T_0z_new)

T_yw_old = np.random.rand(a, a, i, i)
T_yw_new = T_yw_old.reshape(a, a)
T_yw_new_sp = sp.csr_matrix(T_yw_new)

In [23]:
R_old += oe.contract("klcd,acik,bdjl->abij", V_klcd_old, T_0z_old, T_yw_old)
R_new += oe.contract("cd,ac,bd->ab", V_klcd_new, T_0z_new, T_yw_new)
R_new_dot += T_0z_new @ V_klcd_new @ T_yw_new.T
R_new_dot_sp += (T_0z_new_sp @ V_klcd_new_sp @ T_yw_new_sp.T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 3.552713678800501e-15


## Residual double sym 1.

Term 1.

In [24]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

h_pa = np.random.rand(p, a)
h_pa_new = h_pa
h_pa_new_sp = sp.csr_matrix(h_pa)

T_xy_old = np.random.rand(a, a, i, i)
T_xy_new = T_xy_old.reshape(a, a)
T_xy_new_sp = sp.csr_matrix(T_xy_new)

In [25]:
R_old += oe.contract("ap,pc,cbij->abij", A_old, h_pa, T_xy_old)
R_new += oe.contract("ap,pc,cb->ab", A_old, h_pa, T_xy_new)
R_new_dot += A_old @ h_pa_new @ T_xy_new
R_new_dot_sp += (A_old_sp @ h_pa_new_sp @ T_xy_new_sp).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 3.552713678800501e-15 

Dot Sparse:
Equal: True
Max absolute difference: 3.552713678800501e-15


Term 2.

In [26]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

h_ip_old = np.random.rand(i, p)
h_ip_new = h_ip_old.reshape(p)
h_ip_new_sp = sp.csr_matrix(h_ip_new)

T_xy_old = np.random.rand(a, a, i, i)
T_xy_new = T_xy_old.reshape(a, a)
T_xy_new_sp = sp.csr_matrix(T_xy_new)

In [27]:
R_old -= oe.contract("abkj,kp,pi->abij", T_xy_old, h_ip_old, B_old)
R_new -= oe.contract("ab,p,p->ab", T_xy_new, h_ip_new, B_new)
R_new_dot -= T_xy_new * (h_ip_new @ B_new)
R_new_dot_sp -= (T_xy_new_sp * (h_ip_new_sp @ B_new_sp.T)[0,0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0


Term 3.

In [28]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_xz_old = np.random.rand(a, a, i, i).astype(np.float64)
T_xz_new = T_xz_old.reshape(a, a)
T_xz_new_sp = sp.csr_matrix(T_xz_new)

V_ipap_old = np.random.rand(i, p, a, p).astype(np.float64)
V_ipap_new = V_ipap_old.reshape(p, a, p)
V_ipap_new_sp = sp.csr_matrix(V_ipap_new.reshape(p*a, p))

In [29]:
R_old += oe.contract("acik,krcs,br,sj->abij", T_xz_old, V_ipap_old, A_old, B_old)
R_new += oe.contract("ac,rcs,br,s->ab", T_xz_new, V_ipap_new, A_old, B_new)
R_new_dot += T_xz_new @ (A_old @ (V_ipap_new.reshape(p*a, p) @ B_new).reshape(p, a)).T
R_new_dot_sp += (T_xz_new_sp @ (A_old_sp @ (V_ipap_new_sp @ B_new_sp.T).reshape(p, a)).T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 1.4210854715202004e-14 

Dot Sparse:
Equal: True
Max absolute difference: 1.4210854715202004e-14


Term 4.

In [30]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_cb_old = np.random.rand(a, a, i, i)
T_cb_new = T_cb_old.reshape(a, a)
T_cb_new_sp = sp.csr_matrix(T_cb_new)

V_piap_old = np.random.rand(p, i, a, p)
V_piap_new = V_piap_old.reshape(p, a, p)
V_piap_new_sp = sp.csr_matrix(V_piap_new.reshape(p*a, p))

In [31]:
R_old += oe.contract("bq,qlds,adij,sl->abij", A_old, V_piap_old, T_cb_old, B_old)
R_new += oe.contract("bq,qds,ad,s->ab", A_old, V_piap_new, T_cb_new, B_new)
R_new_dot += T_cb_new @ (A_old @ (V_piap_new.reshape(p*a, p) @ B_new).reshape(p, a)).T
R_new_dot_sp += (T_cb_new_sp @ (A_old_sp @ (V_piap_new_sp @ B_new_sp.T).reshape(p, a)).T).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 1.4210854715202004e-14 

Dot:
Equal: True
Max absolute difference: 1.4210854715202004e-14 

Dot Sparse:
Equal: True
Max absolute difference: 2.842170943040401e-14


Term 5.

In [32]:
R_new = np.zeros((a, a), dtype=np.float64)
R_old = np.zeros((a, a, i, i), dtype=np.float64)
R_new_dot = np.zeros((a, a), dtype=np.float64)
R_new_dot_sp = sp.csr_matrix(np.zeros((a, a), dtype=np.float64))

T_cb_old = np.random.rand(a, a, i, i)
T_cb_new = T_cb_old.reshape(a, a)
T_cb_new_sp = sp.csr_matrix(T_cb_new)

V_iipp_old = np.random.rand(i, i, p, p)
V_iipp_new = V_iipp_old.reshape(p, p)
V_iipp_new_sp = sp.csr_matrix(V_iipp_new)

In [33]:
R_old -= oe.contract("abkj,lkrp,pi,rl->abij", T_cb_old, V_iipp_old, B_old, B_old)
R_new -= oe.contract("ab,rp,p,r->ab", T_cb_new, V_iipp_new, B_new, B_new)
R_new_dot -= T_cb_new *((V_iipp_new @ B_new) @ B_new)
R_new_dot_sp -= (T_cb_new_sp *((V_iipp_new_sp @ B_new_sp.T).T @ B_new_sp.T)[0, 0]).toarray()

print("New Einsum:")
print("Equal:", np.allclose(R_new, R_old.reshape(a, a)))

diff = np.abs(R_new - R_old.reshape(a ,a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot:")
print("Equal:", np.allclose(R_new_dot, R_old.reshape(a, a)))

diff = np.abs(R_new_dot - R_old.reshape(a, a))
max_diff = np.max(diff)
print("Max absolute difference:", max_diff, "\n")

print("Dot Sparse:")
print("Equal:", np.allclose(R_new_dot_sp, R_old.reshape(a, a)))

diff1 = np.abs(R_new_dot_sp - R_old.reshape(a, a))
max_diff = np.max(diff1)
print("Max absolute difference:", max_diff)

New Einsum:
Equal: True
Max absolute difference: 0.0 

Dot:
Equal: True
Max absolute difference: 0.0 

Dot Sparse:
Equal: True
Max absolute difference: 0.0
